In [2]:
import os
import json
import pandas as pd
import traceback
import langchain

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
import PyPDF2

In [63]:
KEY="your_key"


In [39]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [40]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000279DA06B7C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000279DA07ECD0>, temperature=0.5, openai_api_key='sk-proj-Ooh8Xx20b9OWmcMrXbDgT3BlbkFJqCX5JijKKMKidJhr4t4n', openai_proxy='')

In [41]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [42]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [43]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [44]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [45]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not as per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [46]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [47]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [48]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [49]:
file_path=r"D:\mcqgen\experiement\data.txt"


In [50]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [51]:
print(TEXT)


Main menu

WikipediaThe Free Encyclopedia
Search Wikipedia
Search
Create account
Log in

Personal tools
Contents hide
(Top)
History
Syntax
Procedural extensions
Interoperability and standardization
Toggle Interoperability and standardization subsection
Overview
Reasons for incompatibility
Standardization history
Current standard
Alternatives
Distributed SQL processing
Criticisms
Toggle Criticisms subsection
Design
Orthogonality and completeness
Null
Duplicates
Impedance mismatch
SQL data types
Toggle SQL data types subsection
Predefined data types
See also
Notes
References
Sources
External links
SQL

Article
Talk
Read
Edit
View history

Tools
From Wikipedia, the free encyclopedia
This article is about the database language. For other uses, see SQL (disambiguation).
"SEQUEL" redirects here. For the topic of the word, see sequel. For other uses, see Sequel (disambiguation).
SQL (Structured Query Language)
Paradigm	Declarative
Family	Query language
Designed by	Donald D. Chamberlin
Raymon

In [52]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [53]:
NUMBER=5 
SUBJECT="SQL"
TONE="simple"

In [54]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
Main menu

WikipediaThe Free Encyclopedia
Search Wikipedia
Search
Create account
Log in

Personal tools
Contents hide
(Top)
History
Syntax
Procedural extensions
Interoperability and standardization
Toggle Interoperability and standardization subsection
Overview
Reasons for incompatibility
Standardization history
Current standard
Alternatives
Distributed SQL processing
Criticisms
Toggle Criticisms subsection
Design
Orthogonality and completeness
Null
Duplicates
Impedance mismatch
SQL data types
Toggle SQL data types subsection
Predefined data types
See also
Notes
References
Sources
External links
SQL

Article
Talk
Read
Edit
View history

Tools
From Wikipedia, the free encyclopedia
This article is about the database language. For other uses, see SQL (disambiguation).
"SEQUEL" redirects here. For the topic of the word, see sequel. For other uses, see Sequel (disambiguation).
SQL (S

In [55]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:16684
Prompt Tokens:15922
Completion Tokens:762
Total Cost:0.025407000000000002


In [56]:
response

{'text': '\nMain menu\n\nWikipediaThe Free Encyclopedia\nSearch Wikipedia\nSearch\nCreate account\nLog in\n\nPersonal tools\nContents hide\n(Top)\nHistory\nSyntax\nProcedural extensions\nInteroperability and standardization\nToggle Interoperability and standardization subsection\nOverview\nReasons for incompatibility\nStandardization history\nCurrent standard\nAlternatives\nDistributed SQL processing\nCriticisms\nToggle Criticisms subsection\nDesign\nOrthogonality and completeness\nNull\nDuplicates\nImpedance mismatch\nSQL data types\nToggle SQL data types subsection\nPredefined data types\nSee also\nNotes\nReferences\nSources\nExternal links\nSQL\n\nArticle\nTalk\nRead\nEdit\nView history\n\nTools\nFrom Wikipedia, the free encyclopedia\nThis article is about the database language. For other uses, see SQL (disambiguation).\n"SEQUEL" redirects here. For the topic of the word, see sequel. For other uses, see Sequel (disambiguation).\nSQL (Structured Query Language)\nParadigm\tDeclarative

In [57]:
quiz = response.get("quiz")
try:
    quiz = response.get("quiz")
    quiz = json.loads(quiz)
    print(quiz)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
print(quiz)

{'1': {'mcq': 'What does SQL stand for?', 'options': {'a': 'Structured Question Language', 'b': 'Structured Query Language', 'c': 'Sequential Query Language', 'd': 'Systematic Query Language'}, 'correct': 'b'}, '2': {'mcq': 'Which of the following is NOT a type of SQL statement?', 'options': {'a': 'Data Manipulation Language (DML)', 'b': 'Data Query Language (DQL)', 'c': 'Data Transmission Language (DTL)', 'd': 'Data Definition Language (DDL)'}, 'correct': 'c'}, '3': {'mcq': 'When was SQL adopted as a standard by ANSI?', 'options': {'a': '1986', 'b': '1992', 'c': '2003', 'd': '2011'}, 'correct': 'a'}, '4': {'mcq': 'Which company developed one of the first commercially available implementations of SQL in 1979?', 'options': {'a': 'IBM', 'b': 'Microsoft', 'c': 'Oracle', 'd': 'Google'}, 'correct': 'c'}, '5': {'mcq': "What does the term 'Null' represent in SQL?", 'options': {'a': 'A value of 0', 'b': 'An empty string', 'c': 'The absence of a value', 'd': 'A placeholder for future data'}, 'c

In [58]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [59]:
print(quiz_table_data)

[{'MCQ': 'What does SQL stand for?', 'Choices': 'a: Structured Question Language | b: Structured Query Language | c: Sequential Query Language | d: Systematic Query Language', 'Correct': 'b'}, {'MCQ': 'Which of the following is NOT a type of SQL statement?', 'Choices': 'a: Data Manipulation Language (DML) | b: Data Query Language (DQL) | c: Data Transmission Language (DTL) | d: Data Definition Language (DDL)', 'Correct': 'c'}, {'MCQ': 'When was SQL adopted as a standard by ANSI?', 'Choices': 'a: 1986 | b: 1992 | c: 2003 | d: 2011', 'Correct': 'a'}, {'MCQ': 'Which company developed one of the first commercially available implementations of SQL in 1979?', 'Choices': 'a: IBM | b: Microsoft | c: Oracle | d: Google', 'Correct': 'c'}, {'MCQ': "What does the term 'Null' represent in SQL?", 'Choices': 'a: A value of 0 | b: An empty string | c: The absence of a value | d: A placeholder for future data', 'Correct': 'c'}]


In [60]:
quiz=pd.DataFrame(quiz_table_data)


In [61]:
quiz.to_csv("sql_quiz.csv",index=False)


In [62]:
print(quiz)

                                                 MCQ  \
0                           What does SQL stand for?   
1  Which of the following is NOT a type of SQL st...   
2        When was SQL adopted as a standard by ANSI?   
3  Which company developed one of the first comme...   
4        What does the term 'Null' represent in SQL?   

                                             Choices Correct  
0  a: Structured Question Language | b: Structure...       b  
1  a: Data Manipulation Language (DML) | b: Data ...       c  
2              a: 1986 | b: 1992 | c: 2003 | d: 2011       a  
3      a: IBM | b: Microsoft | c: Oracle | d: Google       c  
4  a: A value of 0 | b: An empty string | c: The ...       c  
